<a href="https://colab.research.google.com/github/szucshey/osm-data-exploration/blob/main/census/census_data_in_hungary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install folium osm2geojson geopandas


: 

In [ ]:
import requests
import json
#Magyarország településhatárainak, megyehatárainak, a főváros kerületeinek határainak lekérése az OSM API-n keresztül
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
    [out:json];
    rel["wikipedia"~"^hu:"]
    [admin_level=8]
    [type=boundary]
    [name!~"Balatonakarattya"]
    [boundary=administrative];
    out geom;

    rel["wikipedia"~"^hu:"]
    [admin_level=9]
    [type=boundary]
    [boundary=administrative];
  	out geom;
    """

response = requests.get(overpass_url, params={'data': overpass_query})

#Az API válasz lementése egy json fileba
output = open("towns.json", "w")
json.dump(response.json(), output)
output.close()

######################################################################################################################

overpass_query = """
    [out:json];
    rel["ISO3166-2"~"^HU"]
    [admin_level=6]
    [type=boundary]
    [boundary=administrative];
    out geom;
    
    rel["ISO3166-1"~"^HU"]
    [admin_level=2]
    [type=boundary]
    [boundary=administrative];
    out geom;
    """

response = requests.get(overpass_url, params={'data': overpass_query})

#Az API válasz lementése egy json fileba
output = open("states.json", "w")
json.dump(response.json(), output)
output.close()

: 

In [ ]:
import pandas as pd

df = pd.read_csv("data_filtered.csv", delimiter=";")

nemzetisegek = ["Német", "Szlovák", "Román", "Horvát", "Szerb", "Ukrán", "Szlovén", "Roma", "Egyéb"]
color_codes = ["#FFF712", "#09EC8E", "#D57C1E", "#D57C1E", "#DC1111", "#0BF7FF", "#FF74E8", "#A2059D", "#1E9111"]
ev = 2011

In [ ]:
from shapely import geometry
import random

: 

In [ ]:
import folium
import osm2geojson


fgs = [folium.FeatureGroup(name="<span style=\"color:#FFF712\">Németek</span>", overlay=False, control=True, show=False), 
       folium.FeatureGroup(name="<span style=\"color:#09EC8E\">Szlovákok</span>", overlay=False, control=True, show=False),
       folium.FeatureGroup(name="<span style=\"color:#D57C1E\">Románok</span>", overlay=False, control=True, show=False),
       folium.FeatureGroup(name="<span style=\"color:#000785\">Horvátok</span>", overlay=False, control=True, show=False),
       folium.FeatureGroup(name="<span style=\"color:#DC1111\">Szerbek</span>", overlay=False, control=True, show=False),
       folium.FeatureGroup(name="<span style=\"color:#0BF7FF\">Ukránok</span>", overlay=False, control=True, show=False),
       folium.FeatureGroup(name="<span style=\"color:#FF74E8\">Szlovének</span>", overlay=False, control=True, show=False),
       folium.FeatureGroup(name="<span style=\"color:#A2059D\">Romák</span>", overlay=False, control=True, show=False),
       folium.FeatureGroup(name="<span style=\"color:#1E9111\">Egyéb nemzetiségek</span>", overlay=False, control=True, show=False)]

styles_state_borders = [lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#FFF712", 'weight': 1},
         lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#09EC8E", 'weight': 1},
         lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#D57C1E", 'weight': 1},
         lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#000785", 'weight': 1},
         lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#DC1111", 'weight': 1},
         lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#0BF7FF", 'weight': 1},
         lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#FF74E8", 'weight': 1},
         lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#A2059D", 'weight': 1},
         lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#1E9111", 'weight': 1}]

styles_town_borders = [lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#FFF712", 'weight': 0.3},
         lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#09EC8E", 'weight': 0.3},
         lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#D57C1E", 'weight': 0.3},
         lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#000785", 'weight': 0.3},
         lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#DC1111", 'weight': 0.3},
         lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#0BF7FF", 'weight': 0.3},
         lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#FF74E8", 'weight': 0.3},
         lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#A2059D", 'weight': 0.3},
         lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#1E9111", 'weight': 0.3}]
#A térkép létrehozása foliumban, kezdeti koordináták, zoom-lehetőségek, térképvászon megadása, az attribútum kötelező külső vászon használata esetén
m = folium.Map(location=[47.161, 19.505], zoom_start=7, min_zoom=7, maxBounds=[[40.000, 10.000], [60.000, 30.000]], tiles=None, attr="-", prefer_canvas=True, overlay=True)
#https://{s}.basemaps.cartocdn.com/dark_nolabels/{z}/{x}/{y}{r}.png
style_function = lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'opacity': 0.2, 'stroke': True, 'color': "#fff", 'weight': 1}
data = json.load(open("states.json", "r"))
geojson_states = osm2geojson.json2geojson(data)

g1 = folium.GeoJson(geojson_states, name='base_states', style_function=style_function, smooth_factor=2.0, control=True, overlay=True, show=True)
m.add_child(g1)

style_function = lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'opacity': 0.2, 'stroke': True, 'color': "#fff", 'weight': 0.3}
#A korábban kapott json fájl geojson formátumba való átalakítása
data = json.load(open("towns.json", "r"))
geojson_towns = osm2geojson.json2geojson(data)

g = folium.GeoJson(geojson_towns, name='base_towns', style_function=style_function, smooth_factor=2.0, control=True, overlay=True, show=True)
m.add_child(g)


In [ ]:
def get_random_point_in_polygon(poly):
  
  minx, miny, maxx, maxy = poly.bounds
  while True:
       p = geometry.Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
       if poly.contains(p):
          return p

: 

In [11]:
layers = folium.LayerControl(position='bottomleft')

for n in range(8):
    nemzetiseg = nemzetisegek[n]
    df2 = df[df["Év"] == ev][df["Kategória"] == "A-N-H-K"][["Település", nemzetiseg]]
    fill = color_codes[n]
    fgs[n].add_child(folium.GeoJson(geojson_states, name='states_colored', style_function=styles_town_borders[n], smooth_factor=2.0, control=False))
    fgs[n].add_child(folium.GeoJson(geojson_towns, name='towns_colored', style_function=styles_town_borders[n], smooth_factor=2.0, control=False))
    for i in range(len(geojson_towns['features'])):
        poly = geometry.Polygon([[coord[0], coord[1]] for coord in geojson_towns['features'][i]['geometry']['coordinates'][0][0]]) 
  
        #adott megyén belül minden x (representation) főre generálódik egy pont
        #representation = 1000
        try:
            #if geojson['features'][i]['properties']['tags']['name'] in df['Település'].values:
            geojson_towns['features'][i]['properties']['popup'] = "<b>{0}</b>".format(geojson_towns['features'][i]['properties']['tags']['name']) + ": {0}".format(int(df2[df2["Település"] == geojson_towns['features'][i]['properties']['tags']['name']][nemzetiseg]))
            list(fgs[n]._children.values())[1].add_child(folium.features.GeoJsonTooltip(['popup'], labels=False))
            #fgs[n].geojson.add_child(folium.features.GeoJsonTooltip(['popup'], labels=False))
            
            for j in range(int(df2[df2["Település"] == geojson_towns['features'][i]['properties']['tags']['name']][nemzetiseg])): #// representation):
                point = get_random_point_in_polygon(poly)
                fgs[n].add_child(folium.Circle(location=[point.y, point.x], radius=10, fill_color=fill, fill_opacity=1.0, opacity=0.0))
        except:
            print(geojson_towns['features'][i]['properties']['tags']['name'], df2[df2["Település"] == geojson_towns['features'][i]['properties']['tags']['name']][nemzetiseg])
    
    m.add_child(fgs[n])

m.add_child(layers)

#css_modify = "<style>.leaflet-container {background: #080808}</style>"
#m.get_root().html.add_child(folium.Element(css_modify))

m.save(outfile="map.html")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


Óbánya Series([], Name: Német, dtype: int64)
XXIII. kerület Series([], Name: Német, dtype: int64)
Óbánya Series([], Name: Szlovák, dtype: int64)
XXIII. kerület Series([], Name: Szlovák, dtype: int64)
Óbánya Series([], Name: Román, dtype: int64)
XXIII. kerület Series([], Name: Román, dtype: int64)
Óbánya Series([], Name: Horvát, dtype: int64)
XXIII. kerület Series([], Name: Horvát, dtype: int64)
Óbánya Series([], Name: Szerb, dtype: int64)
XXIII. kerület Series([], Name: Szerb, dtype: int64)
Óbánya Series([], Name: Ukrán, dtype: int64)
XXIII. kerület Series([], Name: Ukrán, dtype: int64)
Óbánya Series([], Name: Szlovén, dtype: int64)
XXIII. kerület Series([], Name: Szlovén, dtype: int64)
Óbánya Series([], Name: Roma, dtype: int64)
XXIII. kerület Series([], Name: Roma, dtype: int64)


In [12]:
from google.colab import files
files.download('map.html') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>